In [1]:
!pip install pathway bokeh --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [9]:
# Import required libraries for data processing and plotting

import numpy as np
import pandas as pd
import pathway as pw
import bokeh.plotting
import panel as pn

from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.layouts import gridplot

output_notebook()


In [10]:
# Load the dataset and convert timestamp columns into a single datetime object for sorting and filtering

df = pd.read_csv("dataset.csv")
df["Timestamp"] = pd.to_datetime(df["LastUpdatedDate"] + " " + df["LastUpdatedTime"], format="%d-%m-%Y %H:%M:%S")
df = df.sort_values(by=["SystemCodeNumber", "Timestamp"]).reset_index(drop=True)


In [11]:
# Map traffic conditions and vehicle types to numeric values for computation
# These values will be used in the demand function

traffic_map = {"low": 0, "medium": 1, "high": 2}
vehicle_weights = {"car": 1.0, "bike": 0.5, "truck": 1.5}

df["TrafficLevel"] = df["TrafficConditionNearby"].map(traffic_map)
df["VehicleTypeWeight"] = df["VehicleType"].map(vehicle_weights)


In [12]:
# Define Model 2: Demand-Based Pricing
# This model computes a demand score using occupancy, queue length, traffic level, special day indicator, and vehicle type
# It then uses the demand score to adjust price with bounds

BASE_PRICE = 10.0
LAMBDA = 0.5

ALPHA = 1.5
BETA = 0.8
GAMMA = 0.6
DELTA = 1.0
EPSILON = 0.5

def apply_model_2(group):
    group = group.copy()

    group["Demand"] = (
        ALPHA * (group["Occupancy"] / group["Capacity"]) +
        BETA * group["QueueLength"] -
        GAMMA * group["TrafficLevel"] +
        DELTA * group["IsSpecialDay"] +
        EPSILON * group["VehicleTypeWeight"]
    )

    min_d = group["Demand"].min()
    max_d = group["Demand"].max()

    group["NormalizedDemand"] = (group["Demand"] - min_d) / (max_d - min_d + 1e-6)

    group["Model2_Price"] = BASE_PRICE * (1 + LAMBDA * group["NormalizedDemand"])
    group["Model2_Price"] = group["Model2_Price"].clip(lower=0.5 * BASE_PRICE, upper=2.0 * BASE_PRICE)
    group["Model2_Price"] = group["Model2_Price"].round(2)

    return group

df = df.groupby("SystemCodeNumber", group_keys=False).apply(apply_model_2)


/tmp/ipython-input-12-3671162029.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("SystemCodeNumber", group_keys=False).apply(apply_model_2)


In [13]:
# Define a helper function to calculate Haversine distance (in kilometers)
# This is used to identify nearby competitor lots based on latitude and longitude

def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0)**2
    return R * 2 * np.arcsin(np.sqrt(a))


In [14]:
# Apply Model 3: Competitive Pricing
# For each lot and time, check nearby lots (within 1km) and their prices
# Adjust price accordingly if your lot is full or cheaper/more expensive than others nearby

def adjust_for_competition(df):
    df = df.copy()
    df["Model3_Price"] = df["Model2_Price"]

    lot_coords = df.groupby("SystemCodeNumber")[["Latitude", "Longitude"]].first().to_dict("index")
    all_ids = df["SystemCodeNumber"].unique()

    for idx, row in df.iterrows():
        lot_id = row["SystemCodeNumber"]
        lat, lon = row["Latitude"], row["Longitude"]
        time = row["Timestamp"]
        occ, cap = row["Occupancy"], row["Capacity"]
        own_price = row["Model2_Price"]

        nearby_prices = []
        for other_id in all_ids:
            if other_id == lot_id:
                continue
            lat2, lon2 = lot_coords[other_id]["Latitude"], lot_coords[other_id]["Longitude"]
            distance = haversine(lat, lon, lat2, lon2)
            if distance <= 1.0:
                match = df[(df["SystemCodeNumber"] == other_id) & (df["Timestamp"] == time)]
                if not match.empty:
                    nearby_prices.append(match["Model2_Price"].values[0])

        if nearby_prices:
            avg_nearby_price = np.mean(nearby_prices)
            if occ / cap > 0.9 and own_price > avg_nearby_price:
                df.at[idx, "Model3_Price"] = max(own_price - 1.0, 0.5 * BASE_PRICE)
            elif own_price < avg_nearby_price:
                df.at[idx, "Model3_Price"] = min(own_price + 0.5, 2.0 * BASE_PRICE)

    return df

df = adjust_for_competition(df)


In [15]:
# Visualize the results using Bokeh
# Create an interactive line chart for each lot showing Model 3 price changes over time

plots = []

for lot in df["SystemCodeNumber"].unique():
    lot_df = df[df["SystemCodeNumber"] == lot]

    source = ColumnDataSource(data={
        "Timestamp": lot_df["Timestamp"],
        "Price": lot_df["Model3_Price"],
        "Occupancy": lot_df["Occupancy"],
        "QueueLength": lot_df["QueueLength"]
    })

    p = bokeh.plotting.figure(
        x_axis_type="datetime", width=400, height=300,
        title=f"Model 3 Price Trend - {lot}"
    )
    p.line("Timestamp", "Price", source=source, line_width=2, color="orange", legend_label="Model3 Price")

    hover = HoverTool(tooltips=[
        ("Time", "@Timestamp{%F %H:%M}"),
        ("Price", "@Price"),
        ("Occupancy", "@Occupancy"),
        ("Queue", "@QueueLength")
    ], formatters={"@Timestamp": "datetime"})

    p.add_tools(hover)
    p.legend.location = "top_left"
    p.xaxis.axis_label = "Time"
    p.yaxis.axis_label = "Price ($)"

    plots.append(p)

show(gridplot(plots, ncols=2))
